/*
 * Copyright 2021 ConsenSys AG.
 *
 * Licensed under the Apache License, Version 2.0 (the "License"); you may 
 * not use this file except in compliance with the License. You may obtain 
 * a copy of the License at http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software dis-
 * tributed under the License is distributed on an "AS IS" BASIS, WITHOUT 
 * WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the 
 * License for the specific language governing permissions and limitations 
 * under the License.
 */

# Notebook description

This notebook contains some basic processing to automate the collection of statistics relating to the Dafny files.
By creating functions to perform analysis of Dafny files, additional results can easily be added to the pandas dataframe.
The use of a pandas dataframe provides many options for visualisation and the data can easily by stored in a csv.
The data can also easily be supplemented with timestamps to faciliate time series analysis.

This file is a working file and will be converted to a python script in due course.

# TODO: Reformat function documentation to standard style used within this repo

## Libraries

In [2]:
# import libraries
import os
import subprocess

import pandas as pd
import re
import numpy as np

import time
import shutil

## File processing functions

In [3]:
# find *.dfy files, within a given local repo path
# this function will search all subfolders of dirName
# a sorted list of files is returned
def getListOfDafnyFiles(dirName,exclude_folders=[]):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        # if entry is a directory then append the list of files in this directory to allFiles
        if os.path.isdir(fullPath):
            if os.path.abspath(fullPath) not in exclude_folders:
                allFiles = allFiles + getListOfDafnyFiles(fullPath, exclude_folders)
        # else append file only if it is a Dafny file
        else:
            if entry.endswith(".dfy"):
                allFiles.append(fullPath)
    return sorted(allFiles)   

In [4]:
# find folders within the repo that have *.dfy files
# a sorted list of folders is returned (i.e. full path of each folder)
def getListOfDafnyFolders(dafnyFiles):
    listOfDirectories = list()
    for file in dafnyFiles:
        listOfDirectories.append(os.path.dirname(file))
    return sorted(list(set(listOfDirectories)))

In [5]:
# get folder for an individual dafny file
# i.e. for the full path of a dafny file, the filename and repo path are striped
def getFolder(repo, dafny_file):
    repo_path, folder = os.path.dirname(dafny_file).split(repo,1)
    return folder

### Test file processing functions

In [6]:
# test the getListOfDafnyFiles, getListOfDafnyFolders and getFolder functions
# local repo path needs to be set prior to running the tests
repo_directory = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/"
exclude_folders_rel_path = ["src/dafny/libraries/integers"]
exclude_folders_full_path = [os.path.join(repo_directory,f) for f in exclude_folders_rel_path]


print("Test getListOfDafnyFiles: ")
files = getListOfDafnyFiles(repo_directory, exclude_folders_full_path)
for i in files:
    print(i)
print("Length of returned list: ", len(files))

print("Test getListOfDafnyFolders: ")
directories = getListOfDafnyFolders(files)
for i in directories:
    print(i)
print("Length of returned list: ", len(directories))

print("Test getFolder for each file in files: ")
for file in files:
    print(getFolder(repo_directory, file))

Test getListOfDafnyFiles: 
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/BeaconChainTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/Helpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/attestations/AttestationsHelpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/attestations/AttestationsTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoice.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceHelpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceProofs.p.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoicehelpers.p.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/helpers/Crypto.dfy
/Users/joannefuller/

## Functions to collect statistics

In [7]:
# count the number of "lemmas" in a given dafny file
# this function uses a subprocess call
# an alternative method would be to read and search the file directly
def getLemmas(dafny_file):
    cmd = "cat " + dafny_file +"| grep lemma | wc -l"
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    return result.stdout.strip().decode('ascii')

In [8]:
# count the number of "function methods" in a given dafny file
# this function uses a subprocess call
# an alternative method would be to read and search the file directly
def getFunctions(dafny_file):
    cmd = "cat " + dafny_file +"| grep function | grep method | wc -l"
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    return result.stdout.strip().decode('ascii')

In [9]:
# count the number of ghost (= function and lemmas) processes
# ignores function methods
# to be referred to as "Theorems" in the data display
def getGhost(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    for line in tmp_file.readlines():
        if line.strip().startswith(("function", "lemma")):
            if not line.strip().startswith("function method"):
                count += 1
                #print(line)
    tmp_file.close()
    return count

In [10]:
# count the number of non-ghost ()= function methods and methods and predicates) processes
# to be referred to as "Implementations" in the data display
def getNonGhost(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    for line in tmp_file.readlines():
        if line.strip().startswith(("function method", "method", "predicate")):
            count += 1
            #print(line)

    tmp_file.close()
    return count

In [11]:
# count the number of lines of code
# the count occurs after the dafny file is printed used the compiler
# the count also occurs after this output has been cleaned
def getLoC(dafny_file):
    show_ghost = True
    executable = "dafny"
    args  = [] 
    args += ['/rprint:-']
    args += ["/noAutoReq"]
    args += ["/noVerify"]
    args += ["/nologo"]
    args += ["/env:0"]
    if show_ghost:
        args += ["/printMode:NoIncludes"]
    else:
        args += ["/printMode:NoGhost"]
    args += [dafny_file]
    cmd = ' '.join([executable] + args)
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    output = result.stdout.decode('ascii')
    #print(type(result.stdout.decode('ascii')))
    #print(result.stdout.decode('ascii'))

    #remove this section once code has be tested OR comment out
    #tmp_file = open("tmp.txt", "w")
    #tmp_file.write(result.stdout.decode('ascii'))
    #tmp_file.close()
    ######---------------------

    count = 0
    for line in output.splitlines():
        # clean output i.e. remove comment at start and verifier status
        if line.startswith(("Dafny program verifier finished", "//")):
            #print(i)
            pass
        else:
            if line.strip():
                count += 1
                #print(line)
    #print("#LoC: ", count)
    return count

In [12]:
# count the number of lines included in the license comment
# assumes license comment is at the start of the file and is of format /* ... */
# assumes that it has been confirmed that the file has a license comment
def getLicenseLineCount(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    flag = 0
    for line in tmp_file.readlines():
        tmp_line = line.strip()
        cleaned = ' '.join(i for i in tmp_line.split() if i not in ["//", "/*", "/**", "*", "*/"])
        if (not flag) and (tmp_line.startswith("/*")):
            if cleaned:
                count += 1
            flag = 1
        elif flag:
            if cleaned:
                count += 1
            if tmp_line.startswith("*/"):
                tmp_file.close()
                return count

In [13]:
# count the number of lines of documentation
# don't include license comment or empty comment lines
def getDocumentation(dafny_file):
    tmp_file = open(dafny_file, "r")
    count = 0
    license_flag = 0
    for line in tmp_file.readlines():
        tmp_line = line.strip()
        if tmp_line.startswith(("//", "/*", "/**", "*", "*/")):
            cleaned = ' '.join(i for i in tmp_line.split() if i not in ["//", "/*", "/**", "*", "*/"])
            if cleaned:
                #print(cleaned)
                count += 1
                #print(line)
        if tmp_line.startswith("* Copyright 2021 ConsenSys AG."):
            license_flag = 1

    tmp_file.close()
    if license_flag:
        count -= getLicenseLineCount(dafny_file)
        #print(getLicenseLineCount(dafny_file))
    return count

In [14]:
# count the number of theorems (getGhost) and implementations (getNonGhost) proved
# i.e. check that the number of errors when verified is zero
# TODO: include arguments for getGhost and getNonGhost to reduce duplicate processing
def getProved(dafny_file):
    cmd = "dafny /dafnyVerify:1 /compile:0 " + dafny_file
    result = subprocess.run(['/bin/bash', '-i', '-c', cmd], stdout=subprocess.PIPE)
    output = result.stdout.decode('ascii')
    for line in output.splitlines():
        if line.startswith("Dafny program verifier finished with "):
            # check no errors
            #print(line, re.findall(r'\d+', line)[1], type(re.findall(r'\d+', line)[1]))
            if not int(re.findall(r'\d+', line)[1]):
                return (getGhost(dafny_file) + getNonGhost(dafny_file))
        else:
            pass
    # if the verifier doesn't finish, return -1
    return -1

### Test statistics functions

In [15]:
# test file options:
test_file = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/ssz/BytesAndBits.dfy"
#test_file = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/test/dafny/merkle/Merkleise.test.dfy"
#test_file = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/test/dafny/ssz/BitListSeDes.tests.dfy"
#test_file = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/ssz/BitListSeDes.dfy"
#test_file = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/merkle/Merkleise.dfy"

#print("Lemmas ...")
#print(getLemmas(test_file))

#print("Function methods ...")
#print(getFunctions(test_file))

#print("LoC ...")
#print(getLoC(test_file))

#print("Documentation ...")
#print(getDocumentation(test_file))

print("Proved (verified from compile) ...")
print(getProved(test_file))

#print("Ghost ...")
#rint(getGhost(test_file))

#print("NonGhost ...")
#print(getNonGhost(test_file))

Proved (verified from compile) ...
13


## Collate results into a pandas dataframe

One row per Dafny file.

In [16]:
# create a pandas dataframe to store stats relating to the dafny files
column_list = ['Files', 'Folder', "LoC", 'Theorems', 'Implementations', "Documentation"] #"#Doc/#LoC (%)", 
repo_directory = "/Users/joannefuller/Documents/vscode/eth2.0-dafny/"
#files = getListOfDafnyFiles(repo_directory)

df = pd.DataFrame(columns=column_list)

df.head()

,Files,Folder,LoC,Theorems,Implementations,Documentation


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
Files              0 non-null object
Folder             0 non-null object
LoC                0 non-null object
Theorems           0 non-null object
Implementations    0 non-null object
Documentation      0 non-null object
dtypes: object(6)
memory usage: 0.0+ bytes


In [18]:
# collect data for each dafny file
for file in files:
    loc = getLoC(file)
    print(file)
    ghost = getGhost(file)
    nonghost = getNonGhost(file)
    doc = getDocumentation(file)
    #proved = getProved(file)
    df2 = pd.DataFrame([[os.path.basename(file), 
                        getFolder(repo_directory, file), 
                        loc, #,
                        ghost, 
                        nonghost,
                        doc #,
                        #round(doc/loc * 100),
                        #proved 
                        ]], 
                        columns=column_list)
    df = df.append(df2, ignore_index=True)



/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/BeaconChainTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/Helpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/attestations/AttestationsHelpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/attestations/AttestationsTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoice.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceHelpers.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceProofs.p.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoiceTypes.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/forkchoice/ForkChoicehelpers.p.dfy
/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/beacon/helpers/Crypto.dfy
/Users/joannefuller/Documents/vscode/eth2.0-daf

In [19]:
df.head()

,Files,Folder,LoC,Theorems,Implementations,Documentation
0,BeaconChainTypes.dfy,src/dafny/beacon,53,0,2,177
1,Helpers.dfy,src/dafny/beacon,110,2,10,45
2,AttestationsHelpers.dfy,src/dafny/beacon/attestations,102,4,6,117
3,AttestationsTypes.dfy,src/dafny/beacon/attestations,27,0,0,63
4,ForkChoice.dfy,src/dafny/beacon/forkchoice,3,3,15,175


In [20]:
# create and append totals for numeric columns
totals = pd.DataFrame([["", 
                        "TOTAL", 
                        df['LoC'].sum(), #,
                        df['Theorems'].sum(), 
                        df['Implementations'].sum(),
                        df['Documentation'].sum(),
                        #round(df['Documentation'].sum()/df['#LoC'].sum() * 100),
                        #df['Proved'].sum()
                        ]], 
                        columns=column_list)
df = df.append(totals, ignore_index=True)

# convert numeric columns to int64
#numCols = ['#LoC', 'Theorems', 'Implementations', "Documentation", "#Doc/#LoC", "Proved"]
#df[numCols] = df[numCols].astype("int64")

#display a sample of rows
df.head(len(df))

,Files,Folder,LoC,Theorems,Implementations,Documentation
0,BeaconChainTypes.dfy,src/dafny/beacon,53,0,2,177
1,Helpers.dfy,src/dafny/beacon,110,2,10,45
2,AttestationsHelpers.dfy,src/dafny/beacon/attestations,102,4,6,117
3,AttestationsTypes.dfy,src/dafny/beacon/attestations,27,0,0,63
4,ForkChoice.dfy,src/dafny/beacon/forkchoice,3,3,15,175
5,ForkChoiceHelpers.dfy,src/dafny/beacon/forkchoice,133,6,4,156
6,ForkChoiceProofs.p.dfy,src/dafny/beacon/forkchoice,104,6,0,108
7,ForkChoiceTypes.dfy,src/dafny/beacon/forkchoice,8,0,0,26
8,ForkChoicehelpers.p.dfy,src/dafny/beacon/forkchoice,49,2,1,53
9,Crypto.dfy,src/dafny/beacon/helpers,5,0,1,12


### Alternative format

May be useful for github

In [21]:
from tabulate import tabulate

print(tabulate(df, headers='keys', tablefmt='github'))


|    | Files                           | Folder                                        |   LoC |   Theorems |   Implementations |   Documentation |
|----|---------------------------------|-----------------------------------------------|-------|------------|-------------------|-----------------|
|  0 | BeaconChainTypes.dfy            | src/dafny/beacon                              |    53 |          0 |                 2 |             177 |
|  1 | Helpers.dfy                     | src/dafny/beacon                              |   110 |          2 |                10 |              45 |
|  2 | AttestationsHelpers.dfy         | src/dafny/beacon/attestations                 |   102 |          4 |                 6 |             117 |
|  3 | AttestationsTypes.dfy           | src/dafny/beacon/attestations                 |    27 |          0 |                 0 |              63 |
|  4 | ForkChoice.dfy                  | src/dafny/beacon/forkchoice                   |     3 |          3 |   

## Group data

One row per folder.

In [22]:
# create a pandas dataframe to store stats relating to the dafny files
# stats grouped by folder
folders = ["/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/merkle",
            "/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/ssz",
            "/Users/joannefuller/Documents/vscode/eth2.0-dafny/src/dafny/utils",
            "/Users/joannefuller/Documents/vscode/eth2.0-dafny/test/"]

column_list = ['Folder', '#Files', '#LoC', 'Theorems', 'Implementations', "Documentation", "#Doc/#LoC (%)", "Proved"]
df_grouped = pd.DataFrame(columns=column_list)

for folder in folders:
    files = getListOfDafnyFiles(folder)
    
    nFiles = 0
    nLoc = 0
    nGhost = 0
    nNonGhost = 0
    nDoc = 0
    nProved = 0
    for file in files:
        nFiles += 1
        nLoc += getLoC(file)
        nGhost += getGhost(file)
        nNonGhost += getNonGhost(file)
        nDoc += getDocumentation(file)
        nProved += getProved(file)
    

    df2 = pd.DataFrame([[getFolder("/Users/joannefuller/Documents/vscode/eth2.0-dafny", file), 
                        nFiles, 
                        nLoc ,
                        nGhost, 
                        nNonGhost,
                        nDoc,
                        round(nDoc/nLoc * 100),
                        nProved]], 
                        columns=column_list)
    df_grouped = df_grouped.append(df2, ignore_index=True)

#display a sample of rows
df_grouped.head(len(df_grouped))


,Folder,#Files,#LoC,Theorems,Implementations,Documentation,#Doc/#LoC (%),Proved
0,/src/dafny/merkle,1,502,9,18,158,31,27
1,/src/dafny/ssz,7,1230,23,20,293,24,43
2,/src/dafny/utils,8,822,32,16,317,39,48
3,/test/lowlevel_modules,14,686,3,75,362,53,22


### Print dataframe to .csv, .tex and .pdf

In [23]:
# create filenames that include the current data string
timestr = time.strftime("%Y%m%d")
rawfile = 'data' + timestr + '.csv'
grouped_rawfile = 'dataGrouped' + timestr + '.csv'
filename = 'data' + timestr + '.tex'
pdffile = 'data' + timestr + '.pdf'

# check if data directory already exists and create if necessary
if not os.path.exists('data'):
    os.makedirs('data')

#print to csv file without an index
df.to_csv("data/" + rawfile, index = False)
df_grouped.to_csv("data/" + grouped_rawfile, index = False)

#print to pdf via latex
template = r'''\documentclass[a4paper, 12pt]{{article}}
\usepackage[landscape]{{geometry}}
\usepackage{{booktabs}}
\begin{{document}}
\section*{{https://github.com/PegaSysEng/eth2.0-dafny}}
\subsection*{{Data collected: {}}}
\scriptsize
{}
\vspace{{2em}}
{}
\end{{document}}
'''

with open(filename, 'w') as f:
    f.write(template.format(time.strftime("%Y-%m-%d"), df.to_latex(index=False), df_grouped.to_latex(index=False)))

subprocess.call(['pdflatex', filename])

# remove surplus files and move .csv, .tex and .pdf files to the data folder
os.remove('data' + timestr + '.log')
os.remove('data' + timestr + '.aux')

shutil.move(filename, "data/" + filename)
shutil.move(pdffile, "data/" + pdffile)

'data/data20210512.pdf'